In [1]:
import numpy as np
import scipy.linalg as la

# Проверим для следующей СЛАУ: $Ax=b$

In [2]:
A = np.array([[2., -8, 0, 5], [-9, 9, -7, 6], [-6, 7, 3, 8], [-1, 8, 5, 1]])
b = np.array([-40., -58, -75, -1])
A, b

(array([[ 2., -8.,  0.,  5.],
        [-9.,  9., -7.,  6.],
        [-6.,  7.,  3.,  8.],
        [-1.,  8.,  5.,  1.]]),
 array([-40., -58., -75.,  -1.]))

In [2]:
def LU_decomposition(A):
    n = len(A)
    L = np.eye(n)
    U = np.zeros((n, n))
    
    for i in range(0, n):
        for j in range(0, i):
            L[i, j] = (A[i, j] - np.dot(L[i, 0:j], U[0:j, j])) / U[j, j]
        for j in range(i, n):
            U[i, j] = A[i, j] - np.dot(L[i, 0:i], U[0:i, j])

    return (L, U)

In [4]:
L1, U1 = LU_decomposition(A)  # P^-1A= LU
L1, U1

(array([[ 1.        ,  0.        ,  0.        ,  0.        ],
        [-4.5       ,  1.        ,  0.        ,  0.        ],
        [-3.        ,  0.62962963,  1.        ,  0.        ],
        [-0.5       , -0.14814815,  0.535     ,  1.        ]]),
 array([[  2.        ,  -8.        ,   0.        ,   5.        ],
        [  0.        , -27.        ,  -7.        ,  28.5       ],
        [  0.        ,   0.        ,   7.40740741,   5.05555556],
        [  0.        ,   0.        ,   0.        ,   5.0175    ]]))

# Сравним с функцией из модуля Scipy.linalg

In [5]:
P, L2, U2 = la.lu(A)
P, L2, U2

(array([[0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.]]),
 array([[ 1.        ,  0.        ,  0.        ,  0.        ],
        [ 0.11111111,  1.        ,  0.        ,  0.        ],
        [ 0.66666667,  0.14285714,  1.        ,  0.        ],
        [-0.22222222, -0.85714286,  0.49651972,  1.        ]]),
 array([[-9.        ,  9.        , -7.        ,  6.        ],
        [ 0.        ,  7.        ,  5.77777778,  0.33333333],
        [ 0.        ,  0.        ,  6.84126984,  3.95238095],
        [ 0.        ,  0.        ,  0.        ,  4.65661253]]))

In [3]:
LU_Pivot(A)

(array([[-9.        ,  9.        , -7.        ,  6.        ],
        [ 0.11111111,  7.        ,  5.77777778,  0.33333333],
        [ 0.66666667,  0.14285714,  6.84126984,  3.95238095],
        [-0.22222222, -0.85714286,  0.49651972,  4.65661253]]),
 array([1, 3, 2, 0]))

In [6]:
# LU разложение со сменой строк
import numpy as np
from numpy import abs, argmax, array, zeros, dot

def swap_rows(A, i , j):
    tmp = A[i].copy()
    A[i] = A[j]
    A[j] = tmp
    
def LU_Pivot(A, eps = 1.0e-10):
    
    n = len(A)
    perm = array(range(n))
    P = np.zeros((n,n))
    L = np.eye(n)
    U = np.zeros((n,n))
    
    # Сохраним коэф.нормировки в массив
    s = zeros((n), dtype=float)
    for i in range(n):
        s[i] = max(abs(A[i, :]))
    
    for k in range(0, n-1):
        # Меняем строки при необходимости
        p = int(argmax(abs(A[k:n, k])/s[k:n])) + k
        if abs(A[p, k]) < eps:
            print('Матрица вырождена!')
            return np.zeros(n), np.zeros(n)
        if p != k:
            swap_rows(A, k, p)
            swap_rows(s, k, p)
            swap_rows(perm, k, p)
        # Исключение
    
        for i in range(k+1, n):
            if A[i, k] != 0.:
                l_el = A[i,k]/ A[k,k]
                A[i, k+1:n] = A[i, k+1:n] - l_el * A[k, k+1:n]
                A[i,k] = l_el
                
        for k in range(n):
            P[k, perm[k]] = 1

        for k in range(n):
            L[k + 1 : n, k] = A[k + 1 : n, k]
            U[k, k : n] = A[k, k : n]
            
    return P, L, U, A # возвращаем LU и вектор перестановок

In [7]:
def MyLU_solve(A, b):
    n = len(A)
    x, y = zeros(n), zeros(n)
    P, L, U, matrix = LU_Pivot(A)
    b = P@b.T
    print(b)

    for i in range(n):
        y[i] = b[i] - dot(L[i, :i], y[:i])
    print(y)
    
    x[n-1] = y[n-1] / U[n-1, n-1]
    
    for i in range(n - 2, -1, -1):
        x[i] = (y[i] - dot(U[i, i + 1:], x[i + 1:]))/U[i, i]
        
    return x

In [8]:
A = np.array([[2., -8, 0, 5], [-9, 9, -7, 6], [-6, 7, 3, 8], [-1, 8, 5, 1]])
b = np.array([-40., -58, -75, -1])
MyLU_solve(A, b)

[-58. -41. -75. -41.]
[-58.         -34.55555556 -31.3968254  -67.9187935 ]


array([-13.67264574,  -7.40906826,   3.83707025, -14.58545092])

# LU разложение в библиотеке Sympy 

In [9]:
# Sympy
from sympy import Matrix
import sympy as sm

In [7]:
A = Matrix([[2, -8, 0, 5], [-9, 9, -7, 6], [-6, 7, 3, 8], [-1, 8, 5, 1]])
A

Matrix([
[ 2, -8,  0, 5],
[-9,  9, -7, 6],
[-6,  7,  3, 8],
[-1,  8,  5, 1]])

In [13]:
Ls, Us, Ps = A.LUdecomposition()
Ls

Matrix([
[   1,     0,       0, 0],
[-9/2,     1,       0, 0],
[  -3, 17/27,       1, 0],
[-1/2, -4/27, 107/200, 1]])

In [14]:
Us

Matrix([
[2,  -8,      0,        5],
[0, -27,     -7,     57/2],
[0,   0, 200/27,    91/18],
[0,   0,      0, 2007/400]])

In [15]:
Ps

[]

# Д/З:
# 1) Дописать модернизированную функцию LU_decomposition, которая будет менять строки при не обходимости и возвращать матрицы $L, U, P$
# 2) Написать функцию MyLU_solve(A, b) для решения СЛАУ с применением функции LU_Pivot.
# 3) Написать функцию решения СЛАУ методом простой итерации и/или методом Зейделя.
# Темы на зачёт:
## 1) Численное решение нелинейных уравнений $f(x)=0$ (дихотомия, м. Ньютона) 
## 2) Задачи на целые числа из 1-й недели онлайн курса
## 3) Решение СЛАУ методом Гаусса, LU разложение и решение СЛАУ с помощью LU разложения


In [1]:
import numpy as np

def seidel(a, x ,b):     
    n = len(a)                   
    for j in range(0, n):        
        # временная переменная d для хранения b[j]
        d = b[j]                  
          
        # для вычисления соответствующих xi, yi, zi
        for i in range(0, n):     
            if(j != i):
                d-=a[j][i] * x[i]
        # обновление значений решения    
        x[j] = d / a[j][j]      
    return x    
                 
x = [0.0, 0.0, 0.0, 0.0]                        
#a = np.array([[2., -8, 0, 5], [-9, 9, -7, 6], [-6, 7, 3, 8], [-1, 8, 5, 1]])
a = np.array([[3, 1, 0, 0], [2, 3, 1, 0], [1, 2, 3, 1], [0, 1, 2, 3]])
b = np.array([4., 5, 4, 4])
#b = np.array([-40., -58, -75, -1])
x = seidel(a, x, b)
print(x)

for i in range(0, 25):            
    x = seidel(a, x, b)
    print(x)  
  

[1.3333333333333333, 0.7777777777777778, 0.37037037037037046, 0.8271604938271605]
[1.0740740740740742, 0.8271604938271603, 0.14814814814814817, 0.9588477366255145]
[1.05761316872428, 0.9122085048010972, 0.0530406950160038, 0.9939033683889651]
[1.0292638317329676, 0.962810547172687, 0.01707056851089776, 1.0010161052685058]
[1.0123964842757711, 0.9860454876458533, 0.004832145054672073, 1.001430074081601]
[1.0046515041180488, 0.9952882822364101, 0.0011139524425101133, 1.00082793762619]
[1.0015705725878632, 0.9985816341272544, 0.0001460738438125997, 1.0003754060617067]
[1.0004727886242486, 0.9996361163025634, -4.0142430360804816e-05, 1.0001480561860527]
[1.0001212945658122, 0.9999325177662454, -4.4795428118564e-05, 1.000052357696664]
[1.0000224940779183, 0.9999999357574273, -2.490776314571323e-05, 1.0000166265896213]
[1.0000000214141909, 1.000008288311588, -1.1074875662690312e-05, 1.0000046204799125]
[0.9999972372294706, 1.0000055334722404, -4.308217954681244e-06, 1.0000010276545563]
[0.99

In [30]:
import scipy.linalg as la

a = np.array([[3, 1, 0, 0], 
              [2, 3, 1, 0], 
              [1, 2, 3, 1], 
              [0, 1, 2, 3]])
b = np.array([4., 5, 4, 4])

u = np.hstack((np.zeros(1), a.diagonal(1)))
l1 = np.hstack((a.diagonal(-2), np.zeros(2)))
l2 = np.hstack((a.diagonal(-1), np.zeros(1)))
d = a.diagonal()
ab = np.vstack((u, d, l2, l1))
ab

array([[0., 1., 1., 1.],
       [3., 3., 3., 3.],
       [2., 2., 2., 0.],
       [1., 1., 0., 0.]])

In [32]:
la.solve_banded((2, 1), ab, b)

array([1.00000000e+00, 1.00000000e+00, 9.71445147e-17, 1.00000000e+00])

In [20]:
help(la.solve_banded)

Help on function solve_banded in module scipy.linalg.basic:

solve_banded(l_and_u, ab, b, overwrite_ab=False, overwrite_b=False, debug=None, check_finite=True)
    Solve the equation a x = b for x, assuming a is banded matrix.
    
    The matrix a is stored in `ab` using the matrix diagonal ordered form::
    
        ab[u + i - j, j] == a[i,j]
    
    Example of `ab` (shape of a is (6,6), `u` =1, `l` =2)::
    
        *    a01  a12  a23  a34  a45
        a00  a11  a22  a33  a44  a55
        a10  a21  a32  a43  a54   *
        a20  a31  a42  a53   *    *
    
    Parameters
    ----------
    (l, u) : (integer, integer)
        Number of non-zero lower and upper diagonals
    ab : (`l` + `u` + 1, M) array_like
        Banded matrix
    b : (M,) or (M, K) array_like
        Right-hand side
    overwrite_ab : bool, optional
        Discard data in `ab` (may enhance performance)
    overwrite_b : bool, optional
        Discard data in `b` (may enhance performance)
    check_finite : bo